In [1]:
from __future__ import annotations

In [2]:
class Value:
  def __init__(self, data: float, father: Value = None, mother: Value = None) -> None:
    self.name = ""
    self.data = data
    self.father: Value = father
    self.mother: Value = mother
    self.grad = 0 # gradient w.r.t to the target
    self.traversed = False
    self._backward = lambda: None # calculates gradients of parents
    self._zero_grad = lambda: None # zeros out gradients of parents


  def __repr__(self) -> str:
    return f"Value(data={self.data})"
  
  # when backwards called on new node, want to calculate gradients for parents
  # you know the derivative of the parent node wrt to the child node
  
  def __add__(self, other: Value):
    father = self; mother = other
    child = Value(self.data + other.data, father, mother)
    def _backward():
      dchild_dfather = 1
      dchild_dmother = 1
      father.grad += dchild_dfather * child.grad
      mother.grad += dchild_dmother * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
      mother.grad = 0; mother.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child
  
  def __sub__(self, other: Value):
    return self + (-other)
  
  def __mul__(self, other: Value):
    father = self; mother = other
    child = Value(father.data * mother.data, father, mother)
    def _backward():
      dchild_dfather = mother.data
      dchild_dmother = father.data
      father.grad += dchild_dfather * child.grad
      mother.grad += dchild_dmother * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
      mother.grad = 0; mother.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child
  
  def __truediv__(self, other: Value):
    return self * (other ** -1)
  
  def __neg__(self):
    father = self
    child = Value(-father.data, father)
    def _backward():
      dchild_dfather = -1
      father.grad += dchild_dfather * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child
  
  def __pow__(self, other: float):
    father = self
    child = Value(father.data ** other, father)
    def _backward():
      dchild_dfather = other * father.data ** (other - 1)
      father.grad += dchild_dfather * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child


  def backward(self): # defines this node to be the target. derivative of all nodes in graph wrt to this node. 
    # edge cases to consider: 
    # 1. _backward() should only be called when a node's gradient wrt to target is fully computed. i.e. when all it's children have had their _backward's called. EX: a -> b -> c & a -> c. target order is c, b, a but _backward could be called on a before b. b is a's kid. the gradient of a wrt to c is not fully computed.
    # 2. _backward should only be called once per node. need some kind of cycle detection. a node could have multiple children, so it could be visited multiple times. just tracking count won't be good enough. need to consider if there's a duplicate visit from the same path. 

    # could do reverse dfs. root is last to be marked as traversed. if reversing order, it will be called before any of it's children can be called. dfs traverses all children before marking parent as traversed. reversing this means that parent is marked as traversed before any of it's children. this behavior is desirable for topological sorting. from perspective of root, but can be applied to any node b.c. recursive. 
    # a b c -> c b a. regardless of the path that is taken, b's child, a will always be traversed before b, even if a isn't reached through b. reversing order, a's parents will always be traversed before a.
    # adding a traversed variable, so you don't have to check the list. having extra indicator variables save so much computation. 

    dfs_order = []
    def dfs(node: Value):
      if node.father:
        dfs(node.father)
      if node.mother:
        dfs(node.mother)
      if not node.traversed: # i forgor traversal check
        dfs_order.append(node)
        node.traversed = True
    dfs(self)

    self.grad = 1 # derivative wrt to self is 1
    topological_order = dfs_order[::-1]

    print(dfs_order)
    print(topological_order)

    for node in topological_order:
      node._backward()

  def zero_grad(self): # zeros out gradients of ancestors
    self.grad = 0; self.traversed = False
    def dfs(node: Value):
      if node.father:
        dfs(node.father)
      if node.mother:
        dfs(node.mother)
      node._zero_grad()

In [3]:
a = Value(-6)
b = a**2 - a
b.backward()
a.grad, b.grad

[Value(data=-6), Value(data=36), Value(data=6), Value(data=42)]
[Value(data=42), Value(data=6), Value(data=36), Value(data=-6)]


(-13, 1)

In [4]:
a = Value(2); a.name = "a"
b = Value(3); b.name = "b"
c = a * -b; c.name = "c"
d = c ** 2; d.name = "d"
e = d - c; e.name = "e"
a, b, c, d, e

(Value(data=2), Value(data=3), Value(data=-6), Value(data=36), Value(data=42))

In [5]:
# c.grad = 1 # FUCK ME. THIS WAS THE ISSUE ALL ALONG. Forgot to comment out a line. which then made c gradient 1 higher. i thought the -1 wasn't being applied. it was. 
e.backward()
a.grad, b.grad, c.grad, d.grad, e.grad

# e: 1, d: 1, c: (2*-6 * 1 + -1) = -13, b: 26, a: 39

[Value(data=2), Value(data=3), Value(data=-3), Value(data=-6), Value(data=36), Value(data=6), Value(data=42)]
[Value(data=42), Value(data=6), Value(data=36), Value(data=-6), Value(data=-3), Value(data=3), Value(data=2)]


(39, 26, -13, 1, 1)

In [6]:
# all in all, my reasoning was actually on point. the only real mistake i made was forgetting to add traversed var. 
# the mistake i spent the most time on was just thinking that the -1 wasn't being applied from sub. what happened was that i forgot to uncomment out code that set c.grad to 1, which then meant that the -1 was being applied, but it was 1 + -13 = 12, which effectively canceled it out. i was not expecting the bug to be here and subsequently wasted a ton of time looking elsewhere. 
# the debugger gave me a massive hint of c.grad being 1 when e's _backward() was called. i was very confused and just was looking for the bug in the wrong place. 
# the way i ended up fixing this was through just running the error in isolation with new variables and seeing that the result was correct. that led me to know that my backprop calculations were actually correct and that the error was somewhere else. 

In [29]:
import random

class Neuron:
  def __init__(self, n_inputs: int):
    self.weights = [Value(random.uniform(-1, 1)) for _ in range(n_inputs)]
    self.bias = Value(random.uniform(-1, 1))

  def __call__(self, inputs: list[Value]):
    return sum([weight * input for weight, input in zip(self.weights, inputs)], self.bias)
  
  def get_weights(self) -> list[Value]:
    return self.weights + [self.bias]
  
  def get_gradients(self) -> list[int]:
    return [weight.grad for weight in self.get_weights()]
  
  # implemented zero grad with dfs from target. 
  # def zero_grad(self):
  #   for weight in self.get_weights():
  #     weight.zero_grad()
  
class Layer:
  def __init__(self, n_inputs: int, n_neurons: int):
    self.neurons = [Neuron(n_inputs) for _ in range(n_neurons)]
  
  def __call__(self, inputs: list[Value]):
    return [neuron(inputs) for neuron in self.neurons]
  
  def get_weights(self) -> list[Value]:
    accumulator_list = []
    for neuron in self.neurons:
      accumulator_list += neuron.get_weights()
    return accumulator_list
  
  def get_gradients(self) -> list[int]:
    accumulator_list = []
    for neuron in self.neurons:
      accumulator_list += neuron.get_gradients()
    return accumulator_list
  
  

class MLP:
  def __init__(self, n_inputs: int, layer_neuron_counts: list[int]):
    self.layers = [Layer(n_inputs, layer_neuron_counts[0])]
    for i in range(1, len(layer_neuron_counts)):
      self.layers.append(Layer(layer_neuron_counts[i-1], layer_neuron_counts[i]))

  def __call__(self, inputs: list[Value]):
    for layer in self.layers:
      inputs = layer(inputs)
    return inputs
  
  def get_weights(self) -> list[Value]:
    accumulator_list = []
    for layer in self.layers:
      accumulator_list += layer.get_weights()
    return accumulator_list
  
  def get_gradients(self) -> list[int]:
    accumulator_list = []
    for layer in self.layers:
      accumulator_list += layer.get_gradients()
    return accumulator_list
  
  def calculate_loss(self, inputs: list[list[Value]], targets: list[Value]):
    predictions = [self(input) for input in inputs]

    unnormalized_loss = Value(0)
    for prediction, target in zip(predictions, targets):
      example_loss = Value(0)
      for idx in range(len(prediction)):
        example_loss += (prediction[idx] - target[idx]) ** 2
      unnormalized_loss += example_loss
    loss = unnormalized_loss / Value(len(inputs))
    return loss

In [30]:
xs = [
  [Value(2.0), Value(3.0), Value(-1.0)],
  [Value(3.0), Value(-1.0), Value(0.5)],
  [Value(0.5), Value(1.0), Value(1.0)],
  [Value(1.0), Value(1.0), Value(-1.0)]
]
ys = [[Value(1.0)], [Value(-1.0)], [Value(-1.0)], [Value(1.0)]]

In [179]:
mlp = MLP(3, [4, 4, 1])
loss = mlp.calculate_loss(xs, ys)
loss

Value(data=3.844751607699317)

In [221]:
loss.zero_grad() # resets gradients of ancestors
loss.backward() # accumulates gradients of ancestors
weights = mlp.get_weights()
gradients = mlp.get_gradients()

learning_rate = 0.001
for weight, gradient in zip(weights, gradients):
  weight.data -= learning_rate * gradient

loss = mlp.calculate_loss(xs, ys)
weight_norm = sum([weight.data ** 2 for weight in weights]) ** 1/2
# possible to get into the ping pong situation where gradients keep on increasing over time. keeping track of norm helps to spot when this could be happening. it's interesting to see that there are local minima. 

loss, weight_norm

[Value(data=0), Value(data=0), Value(data=0.1387833774602646), Value(data=-0.8875477433524778), Value(data=1.297160232675587), Value(data=0.40961248932310934), Value(data=0.04971639768825298), Value(data=0.4593288870113623), Value(data=0.006713705915963626), Value(data=-0.30762568312896094), Value(data=-1.0701055413802707), Value(data=-0.46408293852955673), Value(data=-1.409294166235625), Value(data=-1.873377104765182), Value(data=-0.18156690172645187), Value(data=-2.054944006491634), Value(data=-0.9391838267797252), Value(data=-1.2468095099086862), Value(data=-1.610301658821966), Value(data=-1.0380534101139416), Value(data=-2.3083063500526135), Value(data=-3.346359760166555), Value(data=0.5646951500112948), Value(data=-2.7816646101552602), Value(data=-2.0675439057997878), Value(data=-3.314353415708474), Value(data=-1.2838712788805895), Value(data=-2.17458300141711), Value(data=-2.029143480761248), Value(data=-4.203726482178358), Value(data=0.013397173446394859), Value(data=-4.19032930

(Value(data=0.21842761160272967), 7.393109972288752)

In [11]:
loss = Value(0)
for pred, target in zip(preds, ys):
  example_squared_error = Value(0)
  for idx in range(len(pred)): # compare each element
    example_squared_error += (pred[idx] - target[idx]) ** 2
  loss += example_squared_error

loss

Value(data=22.35815533801221)

In [12]:
loss.zero_grad()
loss.backward()
weights = mlp.get_weights()
gradients = mlp.get_gradients()
mlp.get_gradients()

[Value(data=0), Value(data=0), Value(data=-0.2776463718826323), Value(data=-0.7410387421807423), Value(data=-0.9819012350415555), Value(data=0.3967127252271947), Value(data=-0.562332888407163), Value(data=0.26475307992159314), Value(data=2.0), Value(data=0.5295061598431863), Value(data=-0.032826728563976726), Value(data=0.9944833228916576), Value(data=3.0), Value(data=2.983449968674973), Value(data=2.950623240110996), Value(data=-0.9132724376901205), Value(data=-1.0), Value(data=0.9132724376901205), Value(data=3.863895677801117), Value(data=1.5328565843340596), Value(data=0.5509553492925041), Value(data=-0.538693676163783), Value(data=0.4593487379893546), Value(data=0.887681977767989), Value(data=1.775363955535978), Value(data=2.2347126935253323), Value(data=-0.2588482771285201), Value(data=-0.7765448313855603), Value(data=1.458167862139772), Value(data=0.4635067656819687), Value(data=-0.4635067656819687), Value(data=0.9946610964578033), Value(data=-0.5358176425879533), Value(data=0.01

[-4.236725484047048,
 13.561986366730896,
 -10.130119751760713,
 -1.1194449439119314,
 1.7244815749514855,
 -5.520158362214759,
 4.123279860765575,
 0.4556495782456711,
 0.4768683425433067,
 -1.5264812376090295,
 1.1402044890510246,
 0.12600010479362944,
 0.8640120203316122,
 -2.76574899283676,
 2.0658749937603362,
 0.22829279151582982,
 25.10483711536971,
 -14.084542277800693,
 -3.347328853813134,
 10.727983870274262,
 -1.2632741616444014,
 -19.74048357397876,
 11.074984243244293,
 2.632078031486174,
 -8.435648811415897,
 0.9933401568300062,
 11.204326462076445,
 -6.285952345525617,
 -1.4939178884809186,
 4.787915293378038,
 -0.5638011532647766,
 -0.8145532652908025,
 0.45698802385318565,
 0.10860766135807112,
 -0.3480808997629011,
 0.04098828001138356,
 -21.900732561406105,
 33.58585094648602,
 27.743998287225672,
 6.042724159639908,
 1.7047343002969262]

In [61]:
learning_rate = 0.01
for weight in weights:
  weight.data -= learning_rate * weight.grad

In [62]:
preds = [mlp(x) for x in xs]

loss = Value(0)
for pred, target in zip(preds, ys):
  example_squared_error = Value(0)
  for idx in range(len(pred)): # compare each element
    example_squared_error += (pred[idx] - target[idx]) ** 2
  loss += example_squared_error

loss

Value(data=2.0657631306157977)

In [63]:
# not normalizing loss seems to make the gradients larger. checks out since the height of the loss function becomes much larger in general. loss function landscape becomes far steeper. input space does not scale in comparison. you would have to change the learning rate to be much smaller to compensate for the larger gradients. 
# if you change your dataset/batch size, you'd have to change learning rate. kinda fiddly and annoying. makes far more sense to just normalize and change lr much less. 

In [ ]:
# in the current implementation, some unnecessary gradients are calculated. also evaluates how a change in input affects the target node, which is not strictly necessary. also for the normalization factor it does the same, which is also not necessary. 
# all in all, if i were to implement this in C, managing memory would be a pain. sounds like a good excercise to do. :)